In [1]:
!pip install -q polars==0.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 48.4 MB/s eta 0:00:00


In [2]:
VER = 1

In [3]:
import pandas as pd, numpy as np
import pickle, glob, gc
from collections import Counter
import itertools
# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
paths = ['/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/oof/handcrafted6.pqt',
         '/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/oof/handcrafted7.pqt']

In [6]:
import polars as pl
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_parquet(path)
            .with_column(pl.col('labels').str.split(by=' '))
            .with_column(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_column(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_column(pl.col('vote').cast(pl.UInt8))
    )

In [7]:
subs = [read_sub(path) for path in paths]

<ipython-input-6-19365d2ba5f6>:5: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  pl.read_parquet(path)


In [8]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid'])#.join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')

In [9]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right')).alias('vote_sum'))
    .drop(['vote', 'vote_right'])
    .sort(by='vote_sum')
    .reverse()
)

<ipython-input-9-be1a3c985862>:1: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  subs = (subs


In [10]:
%%time
subs = subs.to_pandas()
subs = subs.sort_values(['session_type','vote_sum'],ascending=[True,False])
subs = subs.groupby('session_type').head(20)
val_clicks = subs[subs.session_type.str.contains('clicks')]
val_carts = subs[subs.session_type.str.contains('carts')]
val_orders = subs[subs.session_type.str.contains('orders')]
val_clicks.session_type = val_clicks.session_type.apply(lambda x:x.split('_')[0])
val_carts.session_type = val_carts.session_type.apply(lambda x:x.split('_')[0])
val_orders.session_type = val_orders.session_type.apply(lambda x:x.split('_')[0])
val_clicks['session'] = val_clicks.session_type
val_carts['session'] = val_carts.session_type
val_orders['session'] = val_orders.session_type
val_clicks = val_clicks[['aid','session']].astype({'aid':str,'session':int})
val_carts = val_carts[['aid','session']].astype({'aid':str,'session':int})
val_orders = val_orders[['aid','session']].astype({'aid':str,'session':int})

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:

CPU times: user 5min 55s, sys: 13.9 s, total: 6min 9s
Wall time: 6min 5s


In [11]:
def df_parallelize_run(func, t_split):
    
    num_cores = np.min([N_CORES, len(t_split)])
    pool = Pool(num_cores)
    df = pool.map(func, t_split)
    pool.close()
    pool.join()
    
    return df

In [12]:
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}

valid_labels = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_labels.parquet')

def hits(b):
    # b[0] : session id
    # b[1] : ground truth
    # b[2] : aids prediction 
    return b[0], len(set(b[1]).intersection(set(b[2]))), np.clip(len(b[1]), 0, 20)

def otto_metric_piece(values, typ, verbose=True):
    
    c1 = values
    a = valid_labels.loc[valid_labels['type']==typ].merge(c1, how='left', on=['session'])

    b=[[a0, a1, a2] for a0, a1, a2 in zip(a["session"], a["ground_truth"], a["aid"])]
    c = df_parallelize_run(hits, b)
    c = np.array(c)
    
    recall = c[:,1].sum() / c[:,2].sum()
    
    print('{} recall = {:.5f}'.format(typ ,recall))
    
    return recall

def otto_metric(clicks, carts, orders, verbose = True):
    
    score = 0
    score += weights["clicks"] * otto_metric_piece(clicks, "clicks", verbose = verbose)
    score += weights["carts"] * otto_metric_piece(carts, "carts", verbose = verbose)
    score += weights["orders"] * otto_metric_piece(orders, "orders", verbose = verbose)
    if verbose:
        print('=============')
        print('Overall Recall = {:.5f}'.format(score))
        print('=============')
    
    return score

In [ ]:
%%time
_ = otto_metric(val_clicks, val_carts, val_orders)